In [1]:
import time
start_time = time.time()
import cv2
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# 选择图片和设置路径
target = 2
data_name = ['0618', '0854', '1066'][target - 1]

data_path = f'./Features/{data_name}_HSV3x3_features.npy'  # 训练集
pred_path = f'./result/pred_{data_name}_HSV3x3.png'  # 模型预测结果二值图
img_path = f'../input_data/{data_name}.png'  # 测试集
binary_path = f'../input_data/{data_name}.npy'  # 测试集评估用的二值图
smoothed_path = f'./result/smoothed_{data_name}_HSV3x3.png'  # 光滑处理图片保存

# 加载数据
data = np.load(data_path, allow_pickle=True)  # 加载训练集
features = data[:, :-1]  # 去掉标签列
labels = data[:, -1]  # 标签列

# 划分训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.25, random_state=42)

# 网格搜索
svc_grid = svm.SVC(kernel='rbf', probability=True)
ovo_svc_grid = OneVsOneClassifier(svc_grid)

param_grid = {
    'estimator__C': [100, 10, 1, 0.1, 0.01, 0.001],
    'estimator__gamma': [0.001, 0.01, 0.1, 1, 10],
    'estimator__kernel': ['rbf'],
    'estimator__class_weight': ['balanced']
}

grid_search = GridSearchCV(estimator=ovo_svc_grid, param_grid=param_grid,
                           cv=10, scoring='accuracy', verbose=1)

# 模型训练
grid_search.fit(X_train, y_train)

# 输出最佳参数和最佳分数
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

# 使用最佳参数训练最终模型
clf = grid_search.best_estimator_

# 在验证集上进行预测并做评估
y_val_pred = clf.predict(X_val)
print("Validation dataset Evaluation:")
print(classification_report(y_val, y_val_pred))

# 读入测试集
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  # 将图像从 BGR 转为 HSV
height, width, _ = img.shape

# 准备测试集特征矩阵
test_features = []
for y in range(1, height - 1):
    for x in range(1, width - 1):
        feature_vector = img[y-1:y+2, x-1:x+2].flatten()  # 提取3x3邻域的特征
        test_features.append(feature_vector)

# 转换为NumPy数组
test_features_array = np.array(test_features)

# 在测试集上进行预测
y_test_pred = clf.predict(test_features_array)

# 创建空白图像
segmented_img = np.zeros((height, width), dtype=np.uint8)
segmented_img[1:-1, 1:-1] = y_test_pred.reshape(height - 2, width - 2)

# 对边缘进行处理
segmented_img[0, :] = segmented_img[-1, :] = 0
segmented_img[:, 0] = segmented_img[:, -1] = 0

# 生成二值图
binary_output = np.zeros_like(segmented_img)
binary_output[segmented_img == 1] = 255  # 将道路标签染成白色

# 保存二值化预测图像
cv2.imwrite(pred_path, binary_output)

# 计算和打印测试集上的评价指标
binary_img = cv2.imread(binary_path, cv2.IMREAD_GRAYSCALE)
print("Test dataset Evaluation:")
print(classification_report(binary_img.flatten() // 255, binary_output.flatten() // 255))

# 显示原图和预测结果
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_HSV2RGB))  # 转换回 RGB 显示
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(binary_output, cmap='gray')
plt.axis('off')

plt.show()

# # 形态学开运算平滑
# kernel = np.ones((5, 5), np.uint8)
# smoothed_img = cv2.morphologyEx(binary_output, cv2.MORPH_OPEN, kernel)

# # 平滑后的评估
# original_labels = binary_img.flatten() // 255
# smoothed_pred = smoothed_img.flatten() // 255

# print("Smoothed Image Evaluation:")
# print(classification_report(original_labels, smoothed_pred))

# # 显示和保存平滑后的图像
# plt.figure(figsize=(10, 5))

# plt.subplot(1, 3, 1)
# plt.imshow(binary_img, cmap='gray')
# plt.axis('off')

# plt.subplot(1, 3, 2)
# plt.imshow(binary_output, cmap='gray')
# plt.axis('off')

# plt.subplot(1, 3, 3)
# plt.imshow(smoothed_img, cmap='gray')
# plt.axis('off')

# plt.show()

# cv2.imwrite(smoothed_path, smoothed_img)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")


Fitting 10 folds for each of 30 candidates, totalling 300 fits


/home/crisi/miniconda3/envs/d2l/lib/python3.9/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


KeyboardInterrupt: 